In [1]:
import timeit
%pylab inline
#import iam_hdf5_iterator
import hdf5_iterator
reload(hdf5_iterator)

import logging
logging.getLogger().setLevel(logging.DEBUG)
logging.getLogger('hdf5_iterator').setLevel(logging.INFO)

Populating the interactive namespace from numpy and matplotlib


In [2]:
def plot_shingles(batch, nr_to_plot, nr_per_row=3):
    nr_rows = nr_to_plot // nr_per_row + min(nr_to_plot % nr_per_row, 1)
    for row_i in range(nr_rows):
        for col_i in range(nr_per_row):
            subplot_cell_id = row_i * nr_per_row + col_i + 1
            # handle partial final row case
            if subplot_cell_id > nr_to_plot:
                break
            
            batch_img = batch[subplot_cell_id-1]
            
            plt.subplot(nr_rows, nr_per_row, subplot_cell_id)
            plt.imshow(-batch_img, cmap="gray")
            plt.gca().axes.get_xaxis().set_ticks([])
            plt.gca().axes.get_yaxis().set_ticks([])
            
    
    
def demo_nmec(nmec_hdf5,
        shingle_size=(120,120),
        batch_size=32,
        num_authors=50,
        num_forms_per_author=8,
        stdev_threshold=10.):
    print "NMEC Iteration Demo"
    nmec_pre = hdf5_iterator.nmec_pre
    nmec_post = hdf5_iterator.zero_one
    nmec_iterator = hdf5_iterator.Hdf5MiniBatcher(nmec_hdf5, 
                         num_authors=num_authors, 
                         num_forms_per_author=num_forms_per_author,
                         shingle_dim=shingle_size,
                         batch_size=batch_size,
                         rng_seed=888,
                         preprocess=nmec_pre,
                         postprocess=nmec_post,
                         add_rotation=False,
                         stdev_threshold=stdev_threshold)    
    print "First batch: "
    mb = nmec_iterator.get_train_batch()
    plt.figure(figsize=(9,3))
    plot_shingles(mb[0], 18, 6)
    plt.show()
    print "Next batch: "
    mb2 = nmec_iterator.get_train_batch()
    plt.figure(figsize=(9,3))
    plot_shingles(mb2[0], 18, 6)
    plt.show()

        
def demo_iam(iam_hdf5,
             shingle_size=(120,120),
             batch_size=32,
             num_authors=100,
             num_forms_per_author=9):
    print "Sampling demo."
    print "Each batch should be different within iterations and identical across iterations."
    for iter_i in range(3):
        print "Iteration {0}.................".format(iter_i+1)
        iam_iterator = hdf5_iterator.Hdf5MiniBatcher(iam_hdf5, 
                         num_authors=num_authors, 
                         num_forms_per_author=num_forms_per_author,
                         shingle_dim=shingle_size,
                         batch_size=batch_size,
                         rng_seed=888)
    
        print "First batch: "
        mb = iam_iterator.get_train_batch()
        plt.figure(figsize=(9,3))
        plot_shingles(mb[0], 18, 6)
        plt.show()
        print "Next batch: "
        mb2 = iam_iterator.get_train_batch()
        plt.figure(figsize=(9,3))
        plot_shingles(mb2[0], 18, 6)
        plt.show()
        
    print "No repeats demo"
    print "The probability of a repeated shingle in a single epoch should be very small."
    iam_iterator = hdf5_iterator.Hdf5MiniBatcher(iam_hdf5, 
                     num_authors=num_authors, 
                     num_forms_per_author=num_forms_per_author,
                     shingle_dim=shingle_size,
                     batch_size=batch_size)
    nr_iters = 5
    unique_imgs = set()
    for i in range(nr_iters):
        print "Batch {0}".format(i)
        mb = iam_iterator.get_train_batch()
        plt.figure(figsize=(9,9))
        plot_shingles(mb[0], batch_size, 6)
        plt.show()
        unique_imgs.update([mb[0][img_i].tostring() for img_i in range(batch_size)])
        print "**Unique shingles so far: {0} (should be {1})**".format(len(unique_imgs), 
                                                                       (i+1) * batch_size)

def main():
    #demo_iam("/fileserver/iam/iam-processed/lines/author_lines.hdf5")
    demo_time = timeit.timeit(lambda: demo_nmec("/fileserver/nmec-handwriting/nmec.hdf5"), number=1)
    print "{0} seconds".format(demo_time)
        
    
       
    
    

In [3]:
# Main Test Cell
main()

NMEC Iteration Demo
First batch: 


NameError: global name 'rng' is not defined